In [1]:
from bs4 import BeautifulSoup
from goose3 import Goose
from pprint import pprint
from tqdm import tqdm
import MySQLdb as mdb
import requests
import string
import os
import re

In [2]:
def mesin_cuci(text):
    baca_word = ['Informasi Menarik Terbaru',
                'Membaca:',
                'Baca juga',
                'Baca :',
                'BACA JUGA:',
                'Artikel ini telah tayang di',
                'Baca:',
                'BACA :',
                'Baca Juga:',
                'Baca artikel sumber',
                'Penulis :']

    junk_word = []
    for word in baca_word:
        for i in text.split('.'):
            if i.find(word) >= 0 :
                junk_word.append(i)

    for j in junk_word:
        text = text.replace(j, ' ').replace('..','.').replace('. . ','. ')
        
    return text

In [3]:
def remove_publisher_title(title):
    publisher_title = ['VIVA','Kompas.com','ANTARA News','iNews News','Okezone Lifestyle','Kabar24']
    
    for p in publisher_title:
        title = title.replace(p,'').strip()
    
    return title

In [4]:
def scrap_content(link):
    g = Goose({
                'use_meta_language': False, 
                'target_language':'id',
                'enable_image_fetching': False,
             })
    extract = g.extract(url=link)
    
    title = extract.title
    title = title.replace(" | ","").replace(" - "," ").replace(" – "," ").replace("   "," ").replace("  "," ")
    title = remove_publisher_title(title)
    regex = re.compile("[,\.!?:']")
    title = regex.sub('', title)
    
    content = extract.cleaned_text
    content = content.replace('‘','')
    content = content.replace('\n\n','\n').replace('\n\n','\n').replace('\n',' ').replace('  ',' ')
    content = re.sub(r'[^\x00-\x7F]+', ' ', content)
    content = mesin_cuci(content)
    
    if len(content) > 100:
        content = content
    else:
        content = "Content Not Valid"
    
    result = {
        'title': title,
        'content': content
    }
        
    return(result)

In [5]:
category = [['WORLD','Dunia'],['NATION','Indonesia'],['BUSINESS','Bisnis'],['TECHNOLOGY','Teknologi'],
            ['ENTERTAINMENT','Hiburan'],['SPORTS','Olahraga'],['SCIENCE','Science'],['HEALTH','Kesehatan']]

In [6]:
# for cat in category:
cat1 = 'HEALTH'
cat2 = 'Kesehatan'
print(cat1)

url  = "https://news.google.com/news/headlines/section/topic/{0}.id_id/{1}?ned=id_id&hl=id&gl=ID".format(cat1,cat2)
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
href = soup.find_all("a", {"class" : "nuEeue"})

result = []

for h in tqdm(range(len(href))):
#     try:
    link = href[h]['href']
    content = scrap_content(link)

    res = {
        'url': link,
        'content': content,
        'category': cat1
    }
    result.append(res)
#     except:
#         pass


HEALTH


100%|██████████| 40/40 [00:42<00:00,  1.07s/it]


In [7]:
for r in result[:10]:
    print(r['content']['content'])
    print()

Bisnis.com, JAKARTA -- Bawang putih telah sangat dianggap sebagai makanan yang bermanfaat bagi kesehatan sejak lama, digunakan untuk mengobati penyakit manusia selama ribuan tahun. Namun, cara bawang putih menguntungkan tubuh telah membingungkan para peneliti selama ribuan tahun. Dalam penelitian terbaru yang diterbitkan oleh para ilmuwan dari Universitas Nottingham, para peneliti menyimpulkan bahwa bawang putih sebenarnya dapat mengurangi risiko jenis kanker tertentu, penyakit kardiovaskular dan diabetes tipe 2. Selain itu, cara menyiapkan bawang putih dapat memiliki efek positif untuk kesehatan. Namun, para ilmuwan tidak dapat menentukan metode persiapan mana yang paling efektif. Bawang putih menghasilkan berbagai senyawa sulfur saat disiapkan, entah itu itu dicincang, difermentasi dalam alkohol atau digencet untuk diambil minyaknya. Menurut para peneliti, senyawa belerang ini dapat mempengaruhi "molekul pemberi sinyal gas" seperti oksida nitrat dan hidrogen sulfida yang secara alami

### Save to DB 

In [8]:
host = '127.0.0.1'
user = 'root'
password = 'qwerty'
db = 'entity_determiner'

In [9]:
con = mdb.connect(host, user, password, db, charset='utf8')

for r in tqdm(range(len(result))):
    url = result[r]['url']
    title = result[r]['content']['title']
    content = result[r]['content']['content']
    category = result[r]['category']
    
    sql = '''
    INSERT INTO news
    VALUE (0,'{url}','{title}','{content}','{category}',NOW())
    '''.format(url=url, title=title, content=content, category=category)

    try:
        cur = con.cursor()
        cur.execute(sql)
        con.commit()
    except:
        pass
    
con.close()

100%|██████████| 40/40 [00:00<00:00, 1300.64it/s]
